In [1]:
from torch.utils.data import DataLoader, random_split
from transformers import AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm

from instruct_goose.reward import RewardModel
from instruct_goose.dataset import PairDataset
from instruct_goose.utils import load_yaml

/Users/education/DATA/projects/ai/RLHF/instructGOOSE/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = load_yaml("../configs/sentiment_config.yml")
reward_checkpoint = config["reward_model"]["model_path"]
reward_data_path = config["reward_data"]["data_path"]
batch_size = config["reward_data"]["batch_size"]

In [3]:
reward_dataset = load_dataset(reward_data_path)
tokenizer = AutoTokenizer.from_pretrained(reward_checkpoint)
tokenizer.pad_token = tokenizer.eos_token

Using custom data configuration Dahoas--rm-static-576a4467763bb58a
Found cached dataset parquet (/Users/education/.cache/huggingface/datasets/Dahoas___parquet/Dahoas--rm-static-576a4467763bb58a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 398.49it/s]


In [4]:
small_reward_dataset, _ = random_split(reward_dataset["train"], [10, len(reward_dataset["train"]) - 10])

In [5]:
reward_model = RewardModel(reward_checkpoint)

In [6]:
# pair_dataset = PairDataset(reward_dataset["train"], tokenizer, max_length=1024)
pair_dataset = PairDataset(small_reward_dataset, tokenizer, max_length=1024)

100%|██████████| 10/10 [00:00<00:00, 1234.85it/s]


In [7]:
pair_dataset[0]

(tensor([[48902,    25,  3966,  ..., 50256, 50256, 50256]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0]]),
 tensor([[48902,    25,   314,  ..., 50256, 50256, 50256]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0]]))

In [8]:
dataloader = DataLoader(pair_dataset, batch_size=2, shuffle=True)

In [9]:
for i, batch in tqdm(enumerate(dataloader)):
    chosen_input_ids, chosen_attention_mask, rejected_input_ids, rejected_attention_mask = batch
    
    chosen_rewards = reward_model(chosen_input_ids, chosen_attention_mask)
    rejected_rewards = reward_model(rejected_input_ids, rejected_attention_mask)
    
    pass

5it [01:13, 14.73s/it]


In [14]:
chosen_rewards

tensor([[2.4260, 8.0676, 6.4001,  ..., 8.4922, 6.5932, 1.5312],
        [2.3436, 7.9532, 8.3668,  ..., 9.5768, 9.3080, 8.9846]],
       grad_fn=<ReshapeAliasBackward0>)

In [21]:
chosen_rewards.shape

torch.Size([2, 1024])

In [22]:
chosen_rewards[:, -1]

tensor([1.5312, 8.9846], grad_fn=<SelectBackward0>)

In [18]:
import einops
last_token = einops.rearrange(chosen_rewards, "b n -> (b n)")

In [20]:
last_token.shape

torch.Size([2048])

In [26]:
import torch
hiddens = torch.randn([2, 1, 1024, 1])

In [27]:
hiddens.shape

torch.Size([2, 1, 1024, 1])

In [28]:
hiddens[0][0][-1]

tensor([0.0683])

In [29]:
from einops import rearrange

last_hidden = rearrange(hiddens, 'b 1 t 1 -> b t')

In [30]:
last_hidden[0]

tensor([ 0.0197,  1.3598, -0.9530,  ...,  0.3503,  0.0706,  0.0683])

In [15]:
last_hidden.shape

torch.Size([2, 1024])

In [17]:
last_hidden = rearrange(hiddens, 'b 1 t 1 -> b t', t=1)

EinopsError:  Error while processing rearrange-reduction pattern "b 1 t 1 -> b t".
 Input tensor shape: torch.Size([2, 1, 1024, 1]). Additional info: {'t': 1}.
 Shape mismatch, 1024 != 1